# 1. 아파트 전월세 데이터 수집 및 전처리

API 키 받아와 제대로 동작하는지 확인

In [32]:
import requests, json
import pandas as pd

BASE_WITH_KEY = (
    "https://apis.data.go.kr/1613000/RTMSDataSvcAptRent/getRTMSDataSvcAptRent"
    "?serviceKey=%2Brnx%2Bvsl%2BpHXWwa%2FTB4HGdUy0wchXuChUCHCs8r%2BD8n1QggwhJXxI55KzJ2cwwUN%2FxbWIt1d3xq0gvsef6ZOxA%3D%3D"
)

params = {
    "LAWD_CD": "26350",    # 부산 해운대구
    "DEAL_YMD": "202501",  # 2025년 1월
    "_type": "json",
    "pageNo": 1,
    "numOfRows": 1000
}

r = requests.get(BASE_WITH_KEY, params=params, timeout=20)
r.raise_for_status()

# 2) JSON 파싱 후 구조 확인
data = r.json()
body = data.get("response", {}).get("body", {})
items = body.get("items", {}).get("item", [])

print(body.get("totalCount"))

# 3) 판다스 DataFrame으로 '그대로' 보기
df_raw = pd.DataFrame(items)
print("\n=== DataFrame shape ===", df_raw.shape)
print("\n=== 컬럼 목록 ===")
print(df_raw.columns.tolist())

print("\n=== 상위 5행(원본 그대로) ===")
# 가독성을 위해 문자열로 출력
df_raw.head(5)


654

=== DataFrame shape === (654, 17)

=== 컬럼 목록 ===
['aptNm', 'buildYear', 'contractTerm', 'contractType', 'dealDay', 'dealMonth', 'dealYear', 'deposit', 'excluUseAr', 'floor', 'jibun', 'monthlyRent', 'preDeposit', 'preMonthlyRent', 'sggCd', 'umdNm', 'useRRRight']

=== 상위 5행(원본 그대로) ===


,aptNm,buildYear,contractTerm,contractType,dealDay,dealMonth,dealYear,deposit,excluUseAr,floor,jibun,monthlyRent,preDeposit,preMonthlyRent,sggCd,umdNm,useRRRight
0,센텀롯데캐슬2차,2005,,,15,1,2025,"23,000",101.7996,17,1660,0,,,26350,반여동,
1,광하주택2,1988,,,18,1,2025,"6,000",43.9700,2,93,0,,,26350,반송동,
2,해운대경동리인뷰2차,2024,,,4,1,2025,"5,000",84.4354,23,1565,185,,,26350,우동,
3,아시아선수촌아파트,2002,,,15,1,2025,"3,000",59.9200,25,1638,80,,,26350,반여동,
4,센텀리슈빌2단지,2019,,,21,1,2025,"36,330",84.9916,28,1355,0,,,26350,재송동,


부산의 구별 코드를 chatgpt 사용해 정보 서치함

In [33]:
import requests
import pandas as pd

BUSAN_GU_CODES = {
    "26110": "중구",   "26140": "서구",   "26170": "동구",   "26200": "영도구",
    "26230": "부산진구","26260": "동래구", "26290": "남구",   "26320": "북구",
    "26350": "해운대구","26380": "사하구","26410": "금정구","26440": "강서구",
    "26470": "연제구", "26500": "수영구","26530": "사상구","26710": "기장군",
}

api키를 활용할 때는 한번에 한 구, 한 달만 불러오기가 가능하므로 모든 구의 10달치 데이터를 반복해서 불러온 후에 병합함

In [34]:
# ✅ 조회할 월 리스트 (한 번에 여러 달은 불가하므로 월별로 반복)
MONTHS_2025 = [f"2025{m:02d}" for m in range(1, 11)]  # ['202501', ..., '202510']

def fetch_one_month(lawd_cd: str, yyyymm: str, page_size: int = 1000) -> pd.DataFrame:
    """
    특정 구/군(LAWD_CD)과 특정 월(YYYYMM)의 전체 페이지를
    '원본 그대로' 수집하여 DataFrame으로 반환합니다.
    """
    rows, page = [], 1
    while True:
        params = {
            "LAWD_CD": lawd_cd,
            "DEAL_YMD": yyyymm,   # API 특성상 한 번에 한 달만 가능
            "_type": "json",      # JSON 우선 (불가 시 XML 파싱 로직을 추가해도 됨)
            "pageNo": page,
            "numOfRows": page_size,
        }
        r = requests.get(BASE_WITH_KEY, params=params, timeout=20)
        r.raise_for_status()

        data = r.json()
        body = data.get("response", {}).get("body", {}) or {}
        items = body.get("items", {}).get("item", []) or []
        total = int(body.get("totalCount", 0) or 0)

        if not items:
            break

        rows.extend(items)

        # 페이지 종료 조건
        if page * page_size >= total:
            break
        page += 1

    return pd.DataFrame(rows)

불러온 데이터에 구별이름과 년월 컬럼을 추가함

In [35]:
def collect_busan_monthly(month_list) -> pd.DataFrame:
    """
    부산 전 구·군 × 지정 월 리스트 수집 후
    원본 필드 유지 + '구별', '년월'만 추가하여 반환
    """
    frames = []
    for ym in month_list:
        for code, gu_name in BUSAN_GU_CODES.items():
            df_part = fetch_one_month(code, ym)
            if df_part.empty:
                continue
            df_part["구별"] = gu_name
            df_part["년월"] = ym
            frames.append(df_part)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

불러온 데이터를 저장함

In [36]:
""""
if __name__ == "__main__":
    df_raw = collect_busan_monthly(MONTHS_2025)
    print("총 수집 건수:", len(df_raw))
    print("컬럼 예시:", list(df_raw.columns)[:15])
    print(df_raw.head(5).to_string(index=False))

    # CSV 저장 (원본 그대로)
    out_path = "busan_rent_raw_2025_01_to_10.csv"
    df_raw.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("CSV 저장 완료 →", out_path)
    
"""

'"\nif __name__ == "__main__":\n    df_raw = collect_busan_monthly(MONTHS_2025)\n    print("총 수집 건수:", len(df_raw))\n    print("컬럼 예시:", list(df_raw.columns)[:15])\n    print(df_raw.head(5).to_string(index=False))\n\n    # CSV 저장 (원본 그대로)\n    out_path = "busan_rent_raw_2025_01_to_10.csv"\n    df_raw.to_csv(out_path, index=False, encoding="utf-8-sig")\n    print("CSV 저장 완료 →", out_path)\n\n'

In [37]:
df=pd.read_csv("busan_rent_raw_2025_01_to_10.csv")

C:\Users\paint\AppData\Local\Temp\ipykernel_17704\3948714413.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("busan_rent_raw_2025_01_to_10.csv")


In [38]:
df.head()

,aptNm,buildYear,contractTerm,contractType,dealDay,dealMonth,dealYear,deposit,excluUseAr,floor,jibun,monthlyRent,preDeposit,preMonthlyRent,sggCd,umdNm,useRRRight,구별,년월
0,금호,1997,25.03~27.03,신규,12,1,2025,"13,000",59.75,9,161,0,,,26110,영주동,,중구,202501
1,월드밸리,1997,25.02~26.02,신규,18,1,2025,"3,000",37.20,15,2,20,,,26110,부평동2가,,중구,202501
2,코모도에스테이트,2013,25.01~26.01,신규,11,1,2025,500,25.36,4,37-1,38,,,26110,대청동1가,,중구,202501
3,금호,1997,25.03~27.03,신규,12,1,2025,"13,000",59.75,9,161,0,,,26110,영주동,,중구,202501
4,금호,1997,25.03~27.03,신규,12,1,2025,"13,000",59.75,9,161,0,,,26110,영주동,,중구,202501


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49472 entries, 0 to 49471
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   aptNm           49472 non-null  object 
 1   buildYear       49472 non-null  int64  
 2   contractTerm    49472 non-null  object 
 3   contractType    49472 non-null  object 
 4   dealDay         49472 non-null  int64  
 5   dealMonth       49472 non-null  int64  
 6   dealYear        49472 non-null  int64  
 7   deposit         49472 non-null  object 
 8   excluUseAr      49472 non-null  float64
 9   floor           49472 non-null  int64  
 10  jibun           49472 non-null  object 
 11  monthlyRent     49472 non-null  object 
 12  preDeposit      49472 non-null  object 
 13  preMonthlyRent  49472 non-null  object 
 14  sggCd           49472 non-null  int64  
 15  umdNm           49472 non-null  object 
 16  useRRRight      49472 non-null  object 
 17  구별              49472 non-null 

전/월세 구분 피쳐 추가함

In [40]:
import pandas as pd
import numpy as np

# monthlyRent 숫자화
df["_monthlyRent_num"] = pd.to_numeric(df["monthlyRent"], errors="coerce")

# 전세/월세 구분 컬럼 추가
# (NaN은 '미상', 0이면 '전세', 0보다 크면 '월세')
df["전/월세"] = np.where(
    df["_monthlyRent_num"].isna(), "미상",
    np.where(df["_monthlyRent_num"] > 0, "월세", "전세")
)

# 보조 컬럼 정리
df.drop(columns=["_monthlyRent_num"], inplace=True)

컬럼들을 한글로 매칭해 의미 파악을 용이하게 함

In [41]:
import pandas as pd

# df: 위 컬럼들이 들어 있는 원본 DataFrame
# 예: df.info() 에서 보신 그대로의 df

col_map = {
    "aptNm":          "단지명",
    "buildYear":      "건축년도",
    "contractTerm":   "계약기간(개월)",
    "contractType":   "계약구분",
    "dealDay":        "거래일",
    "dealMonth":      "거래월",
    "dealYear":       "거래연도",
    "deposit":        "보증금액",
    "excluUseAr":     "전용면적",
    "floor":          "층",
    "jibun":          "지번",
    "monthlyRent":    "월세금액",
    "preDeposit":     "갱신전_보증금액",
    "preMonthlyRent": "갱신전_월세금액",
    "sggCd":          "지역코드",
    "umdNm":          "법정동",
    "useRRRight":     "대항력여부",
}

# 이미 같은 한글 컬럼이 있으면 충돌 피하기 위해 그 항목은 건너뜁니다.
safe_map = {}
for src, dst in col_map.items():
    if src in df.columns:
        if dst not in df.columns:       # 타깃 한글명이 아직 없을 때만 변경
            safe_map[src] = dst
        else:
            # 이미 dst가 존재하면 스킵 (원본 유지)
            pass

df = df.rename(columns=safe_map)

위도/경도 데이터를 불러오기 위해 컬럼의 타입을 지정하고 주소지 컬럼을 생성함

In [42]:
# string dtype으로 통일
df["구별"] = df["구별"].astype("string")
df["법정동"] = df["법정동"].astype("string")
df["지번"] = df["지번"].astype("string")

# 공백/결측 안전하게 주소지 생성
df_apt = df.assign(
    주소지=(
        df["구별"].fillna("").str.strip() + " " +
        df["법정동"].fillna("").str.strip() + " " +
        df["지번"].fillna("").str.strip()
    ).str.replace(r"\s+", " ", regex=True).str.strip()
)

사용할 컬럼만 지정해 남김

In [43]:
# df_apt 기준으로 원하는 컬럼만 남기기
cols_keep = ["법정동", "지번", "단지명", "거래연도", "전용면적", "구별", "보증금액","월세금액", "주소지","전/월세",'층','년월']

# 지정한 컬럼만 남기고, 순서도 그대로 유지
df_apt = df_apt.loc[:, cols_keep]



고유 주소지를 리스트로 저장함

In [44]:
unique_addresses=df_apt['주소지'].unique()

print(f"고유 주소지 개수: {len(unique_addresses)}개")
unique_addresses[:5]

고유 주소지 개수: 3097개


<StringArray>
['중구 영주동 161', '중구 부평동2가 2', '중구 대청동1가 37-1', '중구 동광동5가 16-11', '중구 영주동 160']
Length: 5, dtype: string

위도 경도 데이터를 수집하기 위해 V-world geocoder 서비스 사용해 api 키 발급받고 매칭함

In [45]:
import os
if not os.path.exists('./data/부산_아파트실거래가_위도경도.csv'):
    import requests
    apiurl = "https://api.vworld.kr/req/address?"

    geo_apt={}
    for address in unique_addresses:
        params = {
            "service": "address",
            "request": "getcoord",
            "crs": "epsg:4326",
            "address": address,
            "format": "json",
            "type": "parcel",
            "key": "3346A0C0-1596-3B19-903C-6CADB520EC4A"
        }
        response = requests.get(apiurl, params=params)
        if response.status_code == 200 and response.json()['response']['status']=='OK':
            point= response.json()['response']['result']['point']
            geo_apt[address]=(point['y'],point['x'])  #위도,경도
    print('처리 전 유일한 주소지의 개수:',len(unique_addresses), end='\r')
    print(f'처리된 주소지 개수: {len(geo_apt)}', end='\r')
    df_apt['위도']=df_apt['주소지'].map(lambda x: geo_apt[x][0] if x in geo_apt else np.nan)
    df_apt['경도']=df_apt['주소지'].map(lambda x: geo_apt[x][1] if x in geo_apt else np.nan)
    df_apt=df_apt.dropna(subset=['위도','경도'])
    df_apt.to_csv('./data/부산_아파트실거래가_위도경도.csv',index=False,encoding='utf-8')
    df_apt=pd.read_csv('./data/부산_아파트실거래가_위도경도.csv',encoding='utf-8')
else:
    df_apt=pd.read_csv('./data/부산_아파트실거래가_위도경도.csv',encoding='utf-8')